# Install necessary libraries

Using PyTorch here because huggingface models and facebook 😛

In [1]:
%pip install -q torch transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Load the dataset

In [2]:
import pandas as pd

dataset = pd.read_json("/content/data.jsonl", lines=True)

In [3]:
cols = dataset.columns
cols

Index(['title', 'input_text', 'target_text', 'compression_ratio',
       'readability_score', 'rouge-1', 'rouge-2', 'rouge-l'],
      dtype='object')

In [4]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/data.jsonl")["train"]

# remove columns we don't need
# for now, we just need input-output pairs
dataset = dataset.remove_columns(["title", "compression_ratio",
                                  "readability_score", "rouge-1",
                                  "rouge-2", "rouge-l"])

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
split_dataset = dataset.train_test_split(test_size=0.1)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 8518
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 947
    })
})


# Tokenization

In [6]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["input_text"],
                             max_length=512,
                             truncation=True,
                             padding='max_length')
    labels = tokenizer(examples["target_text"],
                       max_length=128,
                       truncation=True,
                       padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = split_dataset.map(preprocess_function,
                                batched=True,
                                remove_columns=["input_text", "target_text"])

Map:   0%|          | 0/8518 [00:00<?, ? examples/s]

Map:   0%|          | 0/947 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8518
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 947
    })
})

# BART

In [9]:
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

## Training

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bart-finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    logging_dir="./logs",
    report_to="none"
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

trainer.train()

<ipython-input-11-c6da3437dcb5>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.706800,0.612753
2,0.576900,0.593753
3,0.508400,0.587392


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3195, training_loss=0.6606463150239327, metrics={'train_runtime': 2227.9658, 'train_samples_per_second': 11.47, 'train_steps_per_second': 1.434, 'total_flos': 7790602726932480.0, 'train_loss': 0.6606463150239327, 'epoch': 3.0})

In [12]:
trainer.save_model("./bart-finetuned")
tokenizer.save_pretrained("./bart-finetuned")

('./bart-finetuned/tokenizer_config.json',
 './bart-finetuned/special_tokens_map.json',
 './bart-finetuned/vocab.json',
 './bart-finetuned/merges.txt',
 './bart-finetuned/added_tokens.json')

In [ ]:
!zip -r bart-finetuned.zip bart-finetuned

  adding: bart-finetuned/ (stored 0%)
  adding: bart-finetuned/vocab.json (deflated 68%)
  adding: bart-finetuned/merges.txt (deflated 53%)
  adding: bart-finetuned/checkpoint-901/ (stored 0%)
  adding: bart-finetuned/checkpoint-901/vocab.json (deflated 68%)
  adding: bart-finetuned/checkpoint-901/merges.txt (deflated 53%)
  adding: bart-finetuned/checkpoint-901/optimizer.pt (deflated 9%)
  adding: bart-finetuned/checkpoint-901/training_args.bin (deflated 51%)
  adding: bart-finetuned/checkpoint-901/special_tokens_map.json (deflated 85%)
  adding: bart-finetuned/checkpoint-901/config.json (deflated 63%)
  adding: bart-finetuned/checkpoint-901/tokenizer_config.json (deflated 75%)
  adding: bart-finetuned/checkpoint-901/trainer_state.json (deflated 56%)
  adding: bart-finetuned/checkpoint-901/rng_state.pth (deflated 25%)
  adding: bart-finetuned/checkpoint-901/scheduler.pt (deflated 55%)
  adding: bart-finetuned/checkpoint-901/model.safetensors (deflated 8%)
  adding: bart-finetuned/chec

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!mv bart-finetuned /content/drive/MyDrive/

In [ ]:
!mv bart-finetuned.zip /content/drive/MyDrive/

generate some summaries

In [24]:
model_path = "/content/drive/MyDrive/bart-finetuned"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)

import torch

def summarize_text(text, model, tokenizer, max_length=100):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    input_ids = inputs["input_ids"].to(model.device)

    with torch.no_grad():
        summary_ids = model.generate(
            input_ids,
            max_length=max_length,       # Maximum summary length
            num_beams=5,                 # Beam search for better quality
            early_stopping=True,         # Stop when complete
            repetition_penalty=2.0        # Reduce repetitive phrases
        )

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [25]:
input_text = """
The Eiffel Tower is one of the most famous landmarks in the world. Located in Paris, it was built in 1889 for the World's Fair
and was initially criticized before becoming a global icon of France. Millions of people visit it every year.
"""

summary = summarize_text(input_text, model, tokenizer)
print("Original Text:", input_text)
print("Generated Summary:", summary)

Original Text: 
The Eiffel Tower is one of the most famous landmarks in the world. Located in Paris, it was built in 1889 for the World's Fair 
and was initially criticized before becoming a global icon of France. Millions of people visit it every year.

Generated Summary: 
The Eiffel Tower is one of the most famous landmarks in the world. It was built in 1889 for the World's Fair.
It was criticized before it became a global icon of France.


People go to it every year.
